# Extract relevant OSM Features of Germany, convert the cooridate system to EPSG:25832 and convert to GeoJson with *ogr2ogr*

In [ ]:
# Extrahiere relevante OSM-Features direkt aus der deutschlandweiten OSM-Datei
import os

OSM_FILE = "data/germany-latest.osm.pbf"

# Definition der zu extrahierenden Features und deren Filter
EXTRACTIONS = [
    # (feature_name, filter, add_km2_density, add_population_density)
    ("bio_shops", 'nwr/shop=greengrocer nwr/shop=health_food nwr/shop=farm', False, False),
    ("playgrounds", 'nwr/leisure=playground', False, False),
    ("hotels_hostels", 'nwr/tourism=hotel nwr/tourism=hostel', False, False),
    ("gastronomy", 'nwr/amenity=restaurant nwr/amenity=fast_food nwr/amenity=cafe nwr/amenity=bar', False, False),
    ("fast_food", 'nwr/amenity=fast_food', False, False),
    ("schools", 'nwr/amenity=school', False, False),
    ("kindergartens", 'nwr/amenity=kindergarten', False, False),
    ("green_areas", 'nwr/leisure=park nwr/natural=wood', False, False),
    ("water_bodies", 'nwr/natural=water', False, False),
    ("landuse", 'nwr/landuse=residential nwr/landuse=commercial nwr/landuse=industrial nwr/landuse=farm', False, False),
    ("building_types", 'nwr/building=residential nwr/building=commercial nwr/building=apartments', False, False),
    ("all_buildings", 'nwr/building', False, False),
    ("buildings_with_height_levels", 'nwr/building building:levels building:height', False, False),
    ("health", 'nwr/amenity=doctors nwr/amenity=pharmacy', False, False),
    ("finance", 'nwr/amenity=bank nwr/amenity=atm', False, False),
    ("all_highways", 'w/highway', False, False),
    ("cycleways", 'w/highway=cycleway', False, False)
]

# Extrahiere .osm.pbf-Dateien
for name, filter_str, _, _ in EXTRACTIONS:
    pbf_path = f"data/{name}.osm.pbf"
    if not os.path.exists(pbf_path):
        !osmium tags-filter "$OSM_FILE" {filter_str} -o {pbf_path} --overwrite
        print(f"Extrahiert: {pbf_path}")
    else:
        print(f"Bereits vorhanden: {pbf_path}")

# Konvertiere alle extrahierten .osm.pbf-Dateien zu GeoJSON
PBF_GEOJSON_PAIRS = [
    (f"data/{name}.osm.pbf", f"data/{name}.geojson") for name, _, _, _ in EXTRACTIONS
]

for pbf, geojson in PBF_GEOJSON_PAIRS:
    if not os.path.exists(geojson) and os.path.exists(pbf):
        !ogr2ogr -f GeoJSON {geojson} {pbf} -overwrite -sql "SELECT * FROM points" -t_srs EPSG:25832
        print(f"Konvertiert: {pbf} → {geojson}")
    elif not os.path.exists(pbf):
        print(f"Fehlt: {pbf}")
    else:
        print(f"Bereits vorhanden: {geojson}")


## Imports

In [1]:
import geopandas as gpd
import pandas as pd
import os

## Konfiguration

In [2]:
GEMEINDEN_GPKG = "data/DE_VG5000.gpkg"
LAYER = "v_vg5000_gem"
GEMEINDE_ID_COLUMN = "Gemeindeschlüssel_AGS"

DEUTSCHLAND_ATALAS = "data/DeutschlandAtlasData.csv"

EXTRA_FEATURES = "data/bio_shops.geojson"
EXTRA_FEATURES_NAME = "Bio-Läden"


ADD_KM2_DENSITY_COLUMN = False
ADD_POPULATION_DENSITY_COLUMN = False

# --- Überprüfung der Dateipfade ---
if not os.path.exists(GEMEINDEN_GPKG):
    print(f"Fehler: Das GeoPackage der Gemeinden wurde nicht gefunden unter: {GEMEINDEN_GPKG}")
    print("Bitte passen Sie den Pfad 'GEMEINDEN_GPKG' im Skript an.")
    exit()

if not os.path.exists(EXTRA_FEATURES):
    print(f"Fehler: Die GeoJSON-Datei der Bio-Läden wurde nicht gefunden unter: {EXTRA_FEATURES}")
    print("Stellen Sie sicher, dass Sie 'ogr2ogr -f GeoJSON bio_shops.geojson bio_shops.osm.pbf' ausgeführt haben.")
    exit()

## Verification

In [9]:

try:
    gemeinden_gdf = gpd.read_file(GEMEINDEN_GPKG, layer=LAYER)
    extras_gdf = gpd.read_file(EXTRA_FEATURES)
    
    print(f"Gemeinden geladen. Anzahl der Gemeinden: {len(gemeinden_gdf)}")
    if GEMEINDE_ID_COLUMN not in gemeinden_gdf.columns:
        print(f"Warnung: Die angegebene Gemeinden-ID-Spalte '{GEMEINDE_ID_COLUMN}' wurde nicht gefunden.")
        print(f"Verfügbare Spalten: {gemeinden_gdf.columns.tolist()}")
        print("Bitte korrigieren Sie 'GEMEINDE_ID_COLUMN' im Skript, um Fortzufahren.")
        exit()

except Exception as e:
    print(f"Fehler beim Laden des Gemeinden GeoPackage: {e}")
    exit()

try:
    print(f"Zusätzliche Features geladen. Anzahl der Features: {len(extras_gdf)}")
    # Sicherstellen, dass das CRS übereinstimmt
    if gemeinden_gdf.crs != extras_gdf.crs:
        print(f"CRS-Konflikt! Gemeinden CRS: {gemeinden_gdf.crs}, Zusätzliche Features CRS: {extras_gdf.crs}")
        exit()

except Exception as e:
    print(f"Fehler beim Laden der GeoJSON: {e}")
    exit()


Gemeinden geladen. Anzahl der Gemeinden: 10957
Zusätzliche Features geladen. Anzahl der Features: 7619


In [4]:
gemeinden_gdf

,Objektidentifikator,BeginnLebenszeit,Admin_Ebene_ADE,ADE,Geofaktor_GF,GF,Besondere_Gebiete_BSG,BSG,Regionalschlüssel_ARS,Gemeindeschlüssel_AGS,...,VerwaltungsgemeinschaftTeil1,VerwaltungsgemeinschaftTeil2,Gemeinde,Funk_Schlüsselstelle3,FK_S3,Europ_Statistikschlüssel_NUTS,RegioschlüsselAufgefüllt,GemeindeschlüsselAufgefüllt,Wirksamkeit_WSK,geometry
0,DEBKGVG5000008NT,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,010010000000,01001000,...,00,00,000,Regierungsbezirk,R,DEF01,010010000000,01001000,2008-01-01,"MULTIPOLYGON (((531633.213 6075130.547, 532385..."
1,DEBKGVG5000008NU,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,010020000000,01002000,...,00,00,000,Regierungsbezirk,R,DEF02,010020000000,01002000,2006-01-01,"MULTIPOLYGON (((575109.999 6031870.053, 575829..."
2,DEBKGVG5000008NV,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,010030000000,01003000,...,00,00,000,Regierungsbezirk,R,DEF03,010030000000,01003000,2006-02-01,"MULTIPOLYGON (((621250.836 5983463.205, 620859..."
3,DEBKGVG5000008NW,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,010040000000,01004000,...,00,00,000,Regierungsbezirk,R,DEF04,010040000000,01004000,1970-04-26,"MULTIPOLYGON (((564022.128 6000401.822, 564959..."
4,DEBKGVG5000008NX,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,010510011011,01051011,...,00,11,011,Regierungsbezirk,R,DEF05,010510011011,01051011,2009-01-01,"MULTIPOLYGON (((507163.312 5976634.451, 507870..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10952,DEBKGVG5000008NQ,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,160775051011,16077011,...,50,51,011,Regierungsbezirk,R,DEG0M,160775051011,16077011,2018-07-06,"MULTIPOLYGON (((751999.518 5645268.015, 752651..."
10953,DEBKGVG5000008NR,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,160775051023,16077023,...,50,51,023,Regierungsbezirk,R,DEG0M,160775051023,16077023,2018-07-06,"MULTIPOLYGON (((745945.807 5655517.063, 747488..."
10954,DEBKGVG5000008NS,2019-10-04,Gemeinde,6,mit Struktur,9,Deutschland,1,160775051036,16077036,...,50,51,036,Regierungsbezirk,R,DEG0M,160775051036,16077036,2018-07-06,"MULTIPOLYGON (((743071.244 5654791.729, 744601..."
10955,DEBKGVG5000008MZ,2020-11-08,Gemeinde,6,mit Struktur,9,Deutschland,1,160775052003,16077003,...,50,52,003,Regierungsbezirk,R,DEG0M,160775052003,16077003,2019-01-01,"MULTIPOLYGON (((728904.255 5650186.152, 730841..."


# Adding Deutschland Atalas data

In [12]:
da_df = pd.read_csv(DEUTSCHLAND_ATALAS, sep=';', decimal=',', converters={'ID': str})
gemeinden_gdf["Gemeindeschlüssel_AGS"] = gemeinden_gdf["Gemeindeschlüssel_AGS"].astype(str)
gemeinden_gdf = gemeinden_gdf.merge(da_df, left_on=GEMEINDE_ID_COLUMN, right_on="ID", how="left")
gemeinden_gdf
da_df

,ID,Name,luftrtng,p_poli,p_apo,v_breitb50,v_breitb1000,p_nelade,p_selade,p_harzt,...,kbetr_ue6,kinder_bg,straft,einbr,preis_miet_best,reg_energie,pendel,oenv,p_ozmz_miv,p_ozmz_oev
0,01001000,"Flensburg, Stadt",9.59,3.200000,2.69,99.520715,96.960895,2.841364,5.456455,2.215,...,26.74,21.55,10767.152890,117.774176,6.96,1.009969,11.42,99.5,5.76,12.17
1,01002000,"Kiel, Landeshauptstadt",7.81,3.090000,2.52,98.489685,93.469684,3.211509,4.912096,2.060,...,22.46,24.44,10684.773350,117.069075,7.64,0.106325,15.55,99.7,8.54,19.42
2,01003000,"Luebeck, Hansestadt",5.55,4.220000,3.34,97.180070,88.940276,3.921645,6.579440,2.470,...,14.27,20.98,10322.107340,131.135514,7.47,0.248428,15.56,98.4,8.46,17.56
3,01004000,"Neumuenster, Stadt",8.01,3.080000,2.81,98.309014,95.729833,4.538864,5.322970,2.835,...,20.39,20.73,18117.783200,225.151568,6.22,0.315933,17.15,99.5,4.95,12.70
4,01051001,Albersdorf,7.66,3.840000,2.51,98.013621,86.549376,2.434625,14.107875,2.030,...,12.46,14.75,5401.029190,116.079614,5.67,14.015363,26.67,81.7,17.54,39.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9914,16070058,"Grossbreitenbach, Stadt",7.30,10.059537,5.72,63.157895,34.499151,5.461720,21.779745,4.440,...,12.07,10.91,5622.049899,24.350041,4.75,0.395499,17.37,98.7,19.71,46.54
9915,16071102,Am Ettersberg,7.30,10.059537,4.72,86.662265,0.000000,7.823492,12.037460,4.330,...,10.97,9.58,5450.895058,117.818199,5.45,2.698102,19.61,61.8,14.16,25.74
9916,16071103,Grammetal,7.30,10.059537,8.62,94.392870,1.745266,6.733333,7.994561,3.720,...,10.97,9.58,5450.895058,117.818199,5.88,5.023032,18.32,72.8,11.31,24.44
9917,16073113,"Schwarzatal, Stadt",7.30,10.059537,4.79,63.823351,32.365396,7.168269,14.942308,3.370,...,13.42,10.46,5958.972944,34.484797,5.01,0.768503,19.11,87.9,14.00,25.25


In [ ]:
joined_data = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")

print(f"Räumlicher Join abgeschlossen. Anzahl der gejoiniten Einträge: {len(joined_data)}")

counts = joined_data.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=EXTRA_FEATURES_NAME)

# Fügen Sie die Zählungen wieder dem ursprünglichen Gemeinden-GeoDataFrame hinzu
gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
# Ersetzen Sie NaN-Werte (Gemeinden ohne Läden) durch 0
gemeinden_gdf[EXTRA_FEATURES_NAME] = gemeinden_gdf[EXTRA_FEATURES_NAME].fillna(0).astype(int)

if gemeinden_gdf.crs.is_geographic: # Prüft, ob es ein geografisches CRS ist (wie WGS84 - 4326)
    print("Konvertiere Gemeinden-CRS für genaue Flächenberechnung (z.B. nach EPSG:3035)...")
    gemeinden_gdf_proj = gemeinden_gdf.to_crs(epsg=3035) # ETRS89 / LAEA Europe
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf_proj.geometry.area / 1_000_000 # Fläche in km²
    print("Flächenberechnung in km² durchgeführt.")
else:
    print("Gemeinden-CRS ist bereits projiziert. Berechne Fläche in km²...")
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf.geometry.area / 1_000_000 # Annahme: CRS-Einheit ist Meter
    print("Flächenberechnung in km² durchgeführt.")


In [ ]:
# Liste der zu joinenden GeoJSON-Dateien und deren Spaltennamen
def get_feature_files():
    return [
        (f"data/{name}.geojson", name.replace('_', ' ').title().replace(' ', '_'))
        for name, _, _, _ in EXTRACTIONS
    ]

feature_files = get_feature_files()

for file_path, feature_name in feature_files:
    if not os.path.exists(file_path):
        print(f"Warnung: Datei {file_path} nicht gefunden. Überspringe {feature_name}.")
        gemeinden_gdf[feature_name] = 0
        continue
    try:
        extras_gdf = gpd.read_file(file_path)
        if gemeinden_gdf.crs != extras_gdf.crs:
            extras_gdf = extras_gdf.to_crs(gemeinden_gdf.crs)
        joined = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")
        counts = joined.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=feature_name)
        gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
        gemeinden_gdf[feature_name] = gemeinden_gdf[feature_name].fillna(0).astype(int)
        print(f"{feature_name}: Join und Zählung abgeschlossen.")
    except Exception as e:
        print(f"Fehler beim Join für {feature_name}: {e}")
        gemeinden_gdf[feature_name] = 0


In [ ]:
# Dichte pro km² hinzufügen, wenn gewünscht
if ADD_KM2_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_km2'
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Flaeche_km2']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:
# Dichte pro Einwohner hinzufügen, wenn gewünscht
if ADD_POPULATION_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_Einwohner'
    gemeinden_gdf['Einwohner'] = (
        gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Einwohner']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:

# Dichte pro km² und pro Einwohner für alle Features, die es wünschen
for name, _, add_km2_density, add_population_density in EXTRACTIONS:
    feature_col = name.replace('_', ' ').title().replace(' ', '_')
    if add_km2_density and feature_col in gemeinden_gdf.columns:
        density_col = f'Dichte_{feature_col}_pro_km2'
        gemeinden_gdf[density_col] = (
            gemeinden_gdf[feature_col] / gemeinden_gdf['Flaeche_km2']
        ).replace([float('inf'), -float('inf')], 0).fillna(0)
    if add_population_density and feature_col in gemeinden_gdf.columns:
        density_col = f'Dichte_{feature_col}_pro_Einwohner'
        if 'Einwohner' not in gemeinden_gdf.columns:
            gemeinden_gdf['Einwohner'] = (
                gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])
        gemeinden_gdf[density_col] = (
            gemeinden_gdf[feature_col] / gemeinden_gdf['Einwohner']
        ).replace([float('inf'), -float('inf')], 0).fillna(0)


In [ ]:
import pickle

with open("data/gemeinden_gdf.pkl", "wb") as f:
    pickle.dump(gemeinden_gdf, f)